In [1]:
# Common default imports
from pprint import pprint
from model_manager import LDAManager
from main.lda.config import LdaGeneratorConfig
from gensim import corpora
import pandas as pd
from gensim.corpora import Dictionary
# Plotting utility for the LDA models
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Stop words identified from the next blocks
stop_words = ['game', 'play', '<game_name>']

# Pre Processing Step
We want to process the data for LDA like:
- Keep NOUNS only as they are the elements that mostly hold aspect value
- Split reviews in sentences (Avoid topic modelling in a too broad sense) (https://aclanthology.org/N10-1122.pdf)

In [ ]:
# LDA does not benefit from repeated words among many documents. So we should clean them as they are stopwords.
corpus_path = "../dataset/output/default_sentences/pre_processed.310k.csv"
ds = pd.read_csv(corpus_path)['comments'].apply(lambda x: x.split(' '))

dictionary = corpora.Dictionary(ds)
doc_frequency = {}

for index in range(len(dictionary)):
    doc_frequency[dictionary.get(index)] = len(ds[[dictionary.get(index) in x for x in ds]])
pd.DataFrame.from_dict(doc_frequency, orient='index')[0].map(lambda x: x / len(ds)).sort_values(ascending=False)

Are 'game'(40%), 'play'(20%) and '\<game_name\>'(11%) stopwords? <br>
For sure 'game' is! What about the other two? I believe they bring no context.
We remove them!

### Default: Sentences split

In [6]:
# 80k
corpus_path = "../dataset/output/default_sentences/pre_processed.80k.csv"
test_corpus_path = "../dataset/output/default_sentences/pre_processed.80k.test.csv"

model_manager = LDAManager.from_config(LdaGeneratorConfig(name="default_sentences_80k"), stop_words)
model = model_manager.get_model(corpus_path, True, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

Generating a new compiled model from fs
Model not found. Making a new one.
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/68390 [00:00<?, ?it/s]

{'coherence': [-17.177862702834112,
               -17.02777543298526,
               -17.290263201442876,
               -16.936958294632383],
 'perplexity': -8.588481933552947,
 'topn': [3, 5, 10, 20]}


In [7]:
dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
# Render the plot
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.190883  0.002691       1        1  10.265110
10     0.143829  0.116062       2        1   9.961937
4     -0.048755  0.034062       3        1   7.992255
2     -0.090078  0.035395       4        1   7.733778
13     0.051645  0.040787       5        1   7.196578
11     0.159590  0.037115       6        1   7.019158
5     -0.059096 -0.064933       7        1   6.711771
7     -0.066562  0.026106       8        1   6.662826
0      0.067464 -0.017909       9        1   6.654468
9      0.039191 -0.040358      10        1   6.594026
8     -0.029328 -0.057865      11        1   6.337182
3     -0.049830 -0.063875      12        1   5.786575
12    -0.034708  0.129836      13        1   5.625053
6      0.107520 -0.177115      14        1   5.459282, topic_info=           Term         Freq        Total Category  logprob  loglift
57         card  2241.000000  2241.000000  Default  30.0000  30.0000
258       great  1187.000000  1187.000000  Default  29.0000  29.0000
79         easy   644.000000   644.000000  Default  28.0000  28.0000
195        dice   521.000000   521.000000  Default  27.0000  27.0000
8        player  2470.000000  2470.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
7          love    42.924259   648.353831  Topic14  -5.2775   0.1929
286  experience    38.811085   352.073171  Topic14  -5.3783   0.7027
17         nice    38.186116   538.036268  Topic14  -5.3945   0.2624
33        board    38.142863   624.151115  Topic14  -5.3956   0.1128
118         fun    39.012879  1550.594174  Topic14  -5.3731  -0.7746

[1161 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
111       1  0.042018  <number>
111       2  0.028012  <number>
111       3  0.028012  <number>
111       4  0.084037  <number>
111       5  0.065362  <number>
...     ...       ...       ...
700      12  0.503348     young
3807      2  0.954814   youtube
5852     14  0.904323        yr
7507      5  0.840289      zany
7420      6  0.878548       zee

[4388 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 11, 5, 3, 14, 12, 6, 8, 1, 10, 9, 4, 13, 7])

Now the bigger dataset for 310k

In [2]:
corpus_path = "../dataset/output/default_sentences/pre_processed.310k.csv"
test_corpus_path = "../dataset/output/default_sentences/pre_processed.310k.test.csv"

model_manager = LDAManager.from_config(LdaGeneratorConfig(name="default_sentences"), stop_words)
model = model_manager.get_model(corpus_path, True, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

Generating a new compiled model from fs
Model not found. Making a new one.
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/233897 [00:00<?, ?it/s]

{'coherence': [-14.345296267821746,
               -13.742494545430045,
               -14.779922291032763,
               -15.116752158504827],
 'perplexity': -8.19623024177193,
 'topn': [3, 5, 10, 20]}


In [3]:
dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
# Render the plot
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.231924 -0.006110       1        1  9.162896
12    -0.100420  0.044382       2        1  8.825932
7      0.203365 -0.140904       3        1  8.540160
2      0.080816  0.064884       4        1  8.042860
1     -0.014630  0.184309       5        1  7.608133
8     -0.158234 -0.158961       6        1  7.512413
9      0.036677  0.174920       7        1  7.412632
10    -0.164390 -0.123739       8        1  7.394272
6      0.072340 -0.155483       9        1  6.960229
11     0.064044  0.096032      10        1  6.736107
0     -0.080141  0.073942      11        1  5.758742
5      0.201921 -0.064437      12        1  5.673783
4      0.051864  0.063850      13        1  5.270739
3      0.038715 -0.052686      14        1  5.101103, topic_info=       Term         Freq        Total Category  logprob  loglift
174    card  7263.000000  7263.000000  Default  30.0000  30.0000
147    good  6269.000000  6269.000000  Default  29.0000  29.0000
142    well  2448.000000  2448.000000  Default  28.0000  28.0000
133    time  4748.000000  4748.000000  Default  27.0000  27.0000
209    love  2371.000000  2371.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
840   solid   109.440791   623.082266  Topic14  -5.5117   1.2364
128    make   126.310025  1822.839299  Topic14  -5.3684   0.3063
147    good   129.113918  6269.552848  Topic14  -5.3464  -0.9071
24   design   114.543730  1164.904720  Topic14  -5.4661   0.6563
158    need   115.560843  2214.071117  Topic14  -5.4573   0.0229

[1083 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
694       1  0.056123  <number>
694       2  0.017460  <number>
694       4  0.001247  <number>
694       5  0.710891  <number>
694       6  0.004989  <number>
...     ...       ...       ...
207      11  0.018825      year
207      13  0.935478      year
739      10  0.973116         z
8626     10  0.991476       zee
8117     10  0.923992       zen

[4588 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 13, 8, 3, 2, 9, 10, 11, 7, 12, 1, 6, 5, 4])

Now we check if the NOUN only approach works better. <br>
Take nouns only of the POS tagged ds:

Now that the dataset was built and stored train an LDA on it:

In [ ]:
from main.lda.pre_processing import extract_pos_ds
import pandas as pd

print("Creating the __noun filtered ds:")
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.csv"
store_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

print("Creating the __noun filtered test ds:")
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.test.csv"
store_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.test.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

In [4]:
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.csv"
test_corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.test.csv"

model_manager = LDAManager.from_config(LdaGeneratorConfig(name="default_sentences"), stop_words)
model = model_manager.get_model(corpus_path, False, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

Generating a new compiled model from scratch
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/195789 [00:00<?, ?it/s]

{'coherence': [-8.206318645320582,
               -8.092144166145504,
               -8.572682666308848,
               -9.292421386724754],
 'perplexity': -7.193918552296681,
 'topn': [3, 5, 10, 20]}


In [5]:
dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)

vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.186977 -0.027139       1        1  14.165697
6      0.151868 -0.040687       2        1   9.435898
1      0.104165  0.129464       3        1   8.514341
2     -0.047553  0.033814       4        1   8.384107
5     -0.155608 -0.039763       5        1   7.256713
7      0.017078  0.018696       6        1   7.113230
3      0.000196  0.078796       7        1   6.845400
9     -0.112893 -0.054376       8        1   6.400776
10    -0.085526  0.059666       9        1   6.359796
4     -0.084226  0.107590      10        1   5.981675
12     0.077980  0.076606      11        1   5.432623
8     -0.096928  0.018351      12        1   4.824361
11     0.150969 -0.162208      13        1   4.671468
13    -0.106498 -0.198811      14        1   4.613913, topic_info=          Term          Freq         Total Category  logprob  loglift
9       player  20121.000000  20121.000000  Default  30.0000  30.0000
87        card  17229.000000  17229.000000  Default  29.0000  29.0000
156       dice   4384.000000   4384.000000  Default  28.0000  28.0000
83   expansion   3693.000000   3693.000000  Default  27.0000  27.0000
382     action   4687.000000   4687.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
56       board    159.242220   5019.828217  Topic14  -5.1087  -0.3746
386       feel    128.018954    903.564631  Topic14  -5.3270   1.1219
9       player    167.059296  20121.594383  Topic14  -5.0608  -1.7151
83   expansion    145.204889   3693.798325  Topic14  -5.2010  -0.1602
111        way    141.145856   5081.450331  Topic14  -5.2294  -0.5075

[1133 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
3945      14  0.927588        aa
293        1  0.200203   ability
293        2  0.061386   ability
293        3  0.263682   ability
293        4  0.018137   ability
...      ...       ...       ...
10655      2  0.925629  ziggurat
1723       3  0.931097    zombie
1723       4  0.005060    zombie
1723       8  0.005060    zombie
1723      12  0.055663    zombie

[4946 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 2, 3, 6, 8, 4, 10, 11, 5, 13, 9, 12, 14])

Following the experiments I decided to follow the path of nouns only composition document. <br>
The hyperparameters tuning is now the next step.